In [1]:
import pandas as pd
import numpy as np

## CSV Importation and transformation

We import the csv file

In [3]:
df = pd.read_csv('LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv')
df.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,authorZipCode,QUXVlc3Rpb246MTYy - Quelles sont toutes les choses qui pourraient être faites pour améliorer l'information des citoyens sur l'utilisation des impôts ?,QUXVlc3Rpb246MTYz - Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?,QUXVlc3Rpb246MTY0 - Quels sont selon vous les impôts qu'il faut baisser en priorité ?,"QUXVlc3Rpb246MjA2 - Afin de financer les dépenses sociales, faut-il selon vous...","QUXVlc3Rpb246MjA1 - S'il faut selon vous revoir les conditions d'attribution de certaines aides sociales, lesquelles doivent être concernées ?",QUXVlc3Rpb246MTY1 - Quels sont les domaines prioritaires où notre protection sociale doit être renforcée ?,"QUXVlc3Rpb246MTY2 - Pour quelle(s) politique(s) publique(s) ou pour quels domaines d'action publique, seriez-vous prêts à payer plus d'impôts ?",QUXVlc3Rpb246MTY3 - Y a-t-il d'autres points sur les impôts et les dépenses sur lesquels vous souhaiteriez vous exprimer ?
0,UHJvcG9zYWw6YTY1MmU0YTMtMWUyMC0xMWU5LTk0ZDItZm...,3-2,TVA sociale,2019-01-22 09:35:18,2019-01-22 09:35:18,NaN,False,NaN,VXNlcjo3ZTVjYTUwMi0xZDZlLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,94800,NaN,NaN,NaN,diminuer la CSG de 2 points et en contrepartie...,NaN,NaN,NaN,NaN
1,UHJvcG9zYWw6ZGNhY2VhYmYtMWUyMC0xMWU5LTk0ZDItZm...,3-3,Augmentation du pouvoir d'achat,2019-01-22 09:36:50,2019-01-22 09:38:01,NaN,False,NaN,VXNlcjo5NmNhYWM4ZS0xZTIwLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,27500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Améliorer le pouvoir d’achat en augmentant les...


We correct the date to remove the HH:MM part

In [5]:
def date_curation (date):
    try: 
        return(date[:10])
    except TypeError:
        return(None)

df['createdAt']=df['createdAt'].apply(date_curation)
df['publishedAt']=df['createdAt'].apply(date_curation)
df['updatedAt']=df['updatedAt'].apply(date_curation)

We change the structure of the file to have one row per question answered

In [6]:
df_elastic = pd.DataFrame()
colnumber = 11
for question in (df.columns[colnumber:]):
    df_temp = df.iloc[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, colnumber]].copy().rename(columns={question : 'answer'})
    df_temp["question"]=question[20:]
    df_elastic = pd.concat([df_elastic, df_temp])
    print (question[20:])
    print(colnumber)
    colnumber +=1
df_elastic = df_elastic.reset_index()

Quelles sont toutes les choses qui pourraient être faites pour améliorer l'information des citoyens sur l'utilisation des impôts ?
11
Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?
12
Quels sont selon vous les impôts qu'il faut baisser en priorité ?
13
Afin de financer les dépenses sociales, faut-il selon vous...
14
S'il faut selon vous revoir les conditions d'attribution de certaines aides sociales, lesquelles doivent être concernées ?
15
Quels sont les domaines prioritaires où notre protection sociale doit être renforcée ?
16
Pour quelle(s) politique(s) publique(s) ou pour quels domaines d'action publique, seriez-vous prêts à payer plus d'impôts ?
17
Y a-t-il d'autres points sur les impôts et les dépenses sur lesquels vous souhaiteriez vous exprimer ?
18


## Sentiment analysis component using textblob

In [7]:
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

In [8]:
def sentiment_analysis(expression):
    blob = TextBlob(expression, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
    return (blob.sentiment[0], blob.sentiment[1])

In [9]:
df_elastic['answer'] = df_elastic['answer'].fillna("")
df_elastic['positivity'], df_elastic['intensity'] = zip(*df_elastic['answer'].map(sentiment_analysis))

In [10]:
df_elastic['sentiment'] = df_elastic['positivity']*df_elastic['intensity']

In [10]:
df_elastic.tail()

,index,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,authorZipCode,answer,question,positivity,intensity,sentiment
1014387,126794,UHJvcG9zYWw6MmY0MGJlMmQtNDE4Yi0xMWU5LWJmNTYtZm...,3-143253,SUPPRESSION DS AIDES A LA PRESSE ET SUPPRESSIO...,2019-03-08,2019-03-08,None,False,NaN,VXNlcjowY2E3NDRmZC00MThhLTExZTktYmY1Ni1mYTE2M2...,Citoyen / Citoyenne,69002,comme indiqué dans le titre : suppression des ...,Y a-t-il d'autres points sur les impôts et les...,-0.033333,0.1500,-0.005000
1014388,126795,UHJvcG9zYWw6NzVhM2RkODAtNDE4Yi0xMWU5LWJmNTYtZm...,3-143257,Rapport Cour des Comptes,2019-03-08,2019-03-08,None,False,NaN,VXNlcjpmOTUxNjA5ZC00MTg5LTExZTktYmY1Ni1mYTE2M2...,Citoyen / Citoyenne,92150,,Y a-t-il d'autres points sur les impôts et les...,0.000000,0.0000,0.000000
1014389,126796,UHJvcG9zYWw6YTAyY2VmMTQtNDE4Yi0xMWU5LWJmNTYtZm...,3-143258,Moins d'impôts,2019-03-08,2019-03-08,None,False,NaN,VXNlcjphYmU4OTJjYy0xZWRmLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,75015,,Y a-t-il d'autres points sur les impôts et les...,0.000000,0.0000,0.000000
1014390,126797,UHJvcG9zYWw6YTIyYTgzMzItNDE4Yi0xMWU5LWJmNTYtZm...,3-143259,Argent publique,2019-03-08,2019-03-08,None,False,NaN,VXNlcjowN2RlZThhZS00MTgwLTExZTktYmY1Ni1mYTE2M2...,Citoyen / Citoyenne,3300,la finance ne doit pas dicter nos choix,Y a-t-il d'autres points sur les impôts et les...,0.000000,0.0000,0.000000
1014391,126798,UHJvcG9zYWw6YWVlZWQ1MDMtNDE4Yi0xMWU5LWJmNTYtZm...,3-143262,ISF,2019-03-08,2019-03-08,None,False,NaN,VXNlcjpjMjg0NzUyMi00MTg3LTExZTktYmY1Ni1mYTE2M2...,Citoyen / Citoyenne,2200,Accentuer davantage les contrôles pour supprim...,Y a-t-il d'autres points sur les impôts et les...,0.385000,0.3425,0.131863


In [11]:
df_elastic.to_csv("fiscalite_enriched.csv", encoding='utf-8', index=False)

## Sentiment analysis component using translation and flair

### Translation

Cf master notebook 'democratie' for more detailed info

## Geocoding and source adding

geocoding is a file that has been downloaded using http://www.nosdonnees.fr/wiki/index.php/Image:EUCircos_Regions_departements_circonscriptions_communes_gps.csv.gz

In [13]:
df_elastic_clean = pd.read_csv('fiscalite_enriched.csv', encoding='utf-8', engine='python')

In [14]:
df_elastic_clean.columns

Index(['index', 'id', 'reference', 'title', 'createdAt', 'publishedAt',
       'updatedAt', 'trashed', 'trashedStatus', 'authorId', 'authorType',
       'authorZipCode', 'answer', 'question', 'positivity', 'intensity',
       'sentiment'],
      dtype='object')

In [15]:
try:
    df_elastic = df_elastic_clean.drop(['departement','city','latitude','longitude'], axis=1)
except:
    pass
def code_correction(code):
    if len(code)==4:
        return(str(code[:-2]+'000'))
    if len(code)==6:
        return(str('0'+code[:-2]))
    return(str(code[:-2]))

df_elastic['authorZipCode']=df_elastic['authorZipCode'].astype(str)
df_elastic['authorZipCode']=df_elastic['authorZipCode'].apply(code_correction)

In [16]:
geocoding = pd.read_csv('geocoding_corrected.csv', sep=';')

geocoding['codes_postaux']=geocoding['codes_postaux'].astype(str)

def code_correction(code):
    if len(code)==2:
        return(code+'000')
    if len(code)==4:
        return('0'+code)
    return(code)

geocoding['codes_postaux']=geocoding['codes_postaux'].apply(code_correction)

df_elastic['authorZipCode']=df_elastic['authorZipCode'].astype(str)
geocoding= (geocoding[["nom_departement","nom_commune","codes_postaux","latitude","longitude"]]
            .sort_values(by=["codes_postaux"], axis=0)
            .drop_duplicates(["codes_postaux"], 'first')
            .rename(columns={"nom_departement":"departement", "nom_commune":"city"}))
geocoding=geocoding.fillna(method='pad')
df_elastic = pd.merge(df_elastic, geocoding, how='left', left_on = 'authorZipCode', right_on='codes_postaux')

In [17]:
df_elastic.tail()

,index,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,...,answer,question,positivity,intensity,sentiment,departement,city,codes_postaux,latitude,longitude
1014387,126794,UHJvcG9zYWw6MmY0MGJlMmQtNDE4Yi0xMWU5LWJmNTYtZm...,3-143253,SUPPRESSION DS AIDES A LA PRESSE ET SUPPRESSIO...,2019-03-08,2019-03-08,None,False,NaN,VXNlcjowY2E3NDRmZC00MThhLTExZTktYmY1Ni1mYTE2M2...,...,comme indiqué dans le titre : suppression des ...,Y a-t-il d'autres points sur les impôts et les...,-0.033333,0.1500,-0.005000,NaN,NaN,NaN,NaN,NaN
1014388,126795,UHJvcG9zYWw6NzVhM2RkODAtNDE4Yi0xMWU5LWJmNTYtZm...,3-143257,Rapport Cour des Comptes,2019-03-08,2019-03-08,None,False,NaN,VXNlcjpmOTUxNjA5ZC00MTg5LTExZTktYmY1Ni1mYTE2M2...,...,,Y a-t-il d'autres points sur les impôts et les...,0.000000,0.0000,0.000000,NaN,NaN,NaN,NaN,NaN
1014389,126796,UHJvcG9zYWw6YTAyY2VmMTQtNDE4Yi0xMWU5LWJmNTYtZm...,3-143258,Moins d'impôts,2019-03-08,2019-03-08,None,False,NaN,VXNlcjphYmU4OTJjYy0xZWRmLTExZTktOTRkMi1mYTE2M2...,...,,Y a-t-il d'autres points sur les impôts et les...,0.000000,0.0000,0.000000,NaN,NaN,NaN,NaN,NaN
1014390,126797,UHJvcG9zYWw6YTIyYTgzMzItNDE4Yi0xMWU5LWJmNTYtZm...,3-143259,Argent publique,2019-03-08,2019-03-08,None,False,NaN,VXNlcjowN2RlZThhZS00MTgwLTExZTktYmY1Ni1mYTE2M2...,...,la finance ne doit pas dicter nos choix,Y a-t-il d'autres points sur les impôts et les...,0.000000,0.0000,0.000000,Gironde,Bordeaux,33000,44.833333,-0.566667
1014391,126798,UHJvcG9zYWw6YWVlZWQ1MDMtNDE4Yi0xMWU5LWJmNTYtZm...,3-143262,ISF,2019-03-08,2019-03-08,None,False,NaN,VXNlcjpjMjg0NzUyMi00MTg3LTExZTktYmY1Ni1mYTE2M2...,...,Accentuer davantage les contrôles pour supprim...,Y a-t-il d'autres points sur les impôts et les...,0.385000,0.3425,0.131863,Côtes-d'Armor,Saint-Brieuc,22000,48.516667,-2.783333


In [38]:
df_elastic[df_elastic['latitude'].isna()]["authorZipCode"].value_counts()

0            752
94210        536
37100        528
99000        504
59240        496
87100        472
38920        400
83100        392
72100        384
76100        384
99999        376
13540        352
68200        344
06160        336
76620        304
80090        296
57950        288
06150        264
57970        240
97490        216
66100        208
57155        200
98800        192
13290        184
13080        176
93210        176
37200        176
50130        160
97434        152
97432        152
            ... 
75115          8
02602          8
64890          8
750014         8
689820891      8
44109          8
130007         8
01253          8
100000         8
510000         8
591970         8
06002          8
77652          8
50012          8
1410022        8
49200          8
500            8
50002          8
632000         8
01870          8
76015          8
294800         8
784500         8
564000         8
70015          8
34133          8
07508          8
59034         

In [39]:
df_elastic['source']='Grand debat - fiscalite et depenses publiques'
df_elastic.to_csv("fiscalite_enriched.csv", encoding='utf-8', index=False)

## json writing for elasticsearch

In [40]:
df_elastic = pd.read_csv('fiscalite_enriched.csv', encoding='utf-8', engine='python')

In [41]:
f = open("fiscalite_base_pre_es.json", "w")

for row in df_elastic.iterrows():
    row[1].to_json(f, orient='index', force_ascii=True)
    f.write("\n")
f.write("\n")

1

## Ingestion in elasticsearch

In [ ]:
try:
    df_elastic['trashed'] = np.where((df_elastic['trashed'] =='True') & (df_elastic['trashed'] =='False'),True,False)
except:
    pass

In [42]:
with open('./fiscalite_base_pre_es.json', 'r') as r, open('./fiscalite_base_es.json', 'w') as w:    
    for num, line in enumerate(r):
        if line[0]!='[' and line[0]!=']':
            newline = '{"index":{"_id":'+ '"f_' + str(num)+'"}}'+'\n'
            if line[0]=='\n':
                newline='\n'
            elif line[-2]== ',':
                newline += line[:-2] + "\n" if "\n" in line else line[:-1]
            else:
                newline += line[:-1] + "\n" if "\n" in line else line[:-1]
        else:
            newline = ''               
        w.write(newline)